### 0org+100scaled

In [ ]:
import os
import random
import cv2
import numpy as np
import json

def scale_and_save_images_with_coordinates_and_json(input_dir, output_dir, coordinates=None, class_folders=None):
    # 출력 디렉토리가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 좌표 및 클래스 폴더 정보가 제공되지 않은 경우 예외 처리
    if coordinates is None or class_folders is None:
        raise ValueError("Coordinates and class_folders must be provided.")
    # 캔버스 크기 설정 (기본 640x640)
    canvas_size = (640, 640)
    # 입력 폴더 내의 각 클래스 폴더 처리
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        # 폴더가 아닌 경우 건너뜀
        if os.path.isdir(class_path):
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            print(f"Processing folder: {class_folder}, Total images found: {len(images)}")
            # 클래스 라벨 결정(밥 국 반찬 중)
            if class_folder in class_folders['rice']:
                class_label = 'rice'
            elif class_folder in class_folders['soup']:
                class_label = 'soup'
            elif class_folder in class_folders['side']:
                class_label = 'side'
            else:
                print(f"Skipping unknown class folder: {class_folder}")
                continue

            # 클래스별 출력 폴더 생성
            output_folder = os.path.join(output_dir, class_folder)
            os.makedirs(output_folder, exist_ok=True)

            # 모든 이미지를 선택(100org)
            selected_images = images
            print(f"Selected all {len(selected_images)} images from folder: {class_folder}")

            # 선택된 이미지(모든 이미지)를 하나씩 읽어오기
            for image_file in selected_images:
                image_path = os.path.join(class_path, image_file)
                json_path = os.path.join(class_path, os.path.splitext(image_file)[0] + '.json')

                if not os.path.exists(json_path):
                    print(f"No JSON file for image: {image_file}. Skipping...")
                    continue

                original_image = cv2.imread(image_path)
                if original_image is None:
                    print(f"Error: Failed to read image: {image_path}. Skipping...")
                    continue

                print(f"Processing image: {image_file} with JSON: {json_path}")
                
                # 클래스에 따라 좌표 가져오기
                try:
                    if class_label == 'rice':
                        x, y, w, h = coordinates['rice']
                    elif class_label == 'soup':
                        x, y, w, h = coordinates['soup']
                    elif class_label == 'side':
                        x, y, w, h = random.choice(coordinates['side'])
                    
                    # 캔버스 초기화 (흰 배경)
                    canvas = np.ones((canvas_size[1], canvas_size[0], 3), dtype=np.uint8) * 255
                    # 이미지 크기 조정 및 캔버스에 배치
                    resized_image = cv2.resize(original_image, (w, h), interpolation=cv2.INTER_AREA)
                    canvas[y:y + h, x:x + w] = resized_image

                    # 스케일링된 이미지 이름 생성 및 저장
                    new_image_name = os.path.splitext(image_file)[0] + "_scaled.jpg"
                    new_image_path = os.path.join(output_folder, new_image_name)

                    counter = 1
                    while os.path.exists(new_image_path):
                        new_image_name = f"{os.path.splitext(image_file)[0]}_scaled_{counter}.jpg"
                        new_image_path = os.path.join(output_folder, new_image_name)
                        counter += 1

                    cv2.imwrite(new_image_path, canvas)
                    print(f"Saved scaled image: {new_image_path}")

                    with open(json_path, 'r') as jf:
                        json_data = json.load(jf)

                    transformed_points = []
                    if 'shapes' in json_data and len(json_data['shapes']) > 0:
                        for point in json_data['shapes'][0]['points']:
                            old_x, old_y = point
                            transformed_x = x + (old_x / original_image.shape[1]) * w
                            transformed_y = y + (old_y / original_image.shape[0]) * h
                            transformed_points.append([transformed_x, transformed_y])
                    else:
                        print(f"Skipping JSON file with missing or empty shapes: {json_path}")
                        os.remove(new_image_path)  # JSON 데이터 문제로 이미지 삭제
                        print(f"Deleted scaled image due to missing JSON shapes: {new_image_path}")
                        continue

                    # 변환된 JSON 데이터 생성
                    new_json_data = {
                        "version": json_data.get("version", "0.4.15"),
                        "flags": {},
                        "shapes": [
                            {
                                "label": json_data['shapes'][0]['label'],
                                "text": "",
                                "points": transformed_points,
                                "group_id": None,
                                "shape_type": "polygon",
                                "flags": {}
                            }
                        ],
                        "imagePath": new_image_name,
                        "imageData": None,
                        "imageHeight": canvas_size[1],
                        "imageWidth": canvas_size[0]
                    }

                    # 변환된 JSON 저장
                    new_json_path = os.path.join(output_folder, os.path.splitext(new_image_name)[0] + ".json")
                    with open(new_json_path, 'w') as njf:
                        json.dump(new_json_data, njf, indent=4)
                    print(f"Saved JSON file: {new_json_path}")

                except Exception as e:
                    print(f"Error during processing for {image_file}: {e}")
                    if os.path.exists(new_image_path):
                        os.remove(new_image_path)
                        print(f"Deleted scaled image due to error: {new_image_path}")
                    continue  # 다음 이미지로 

# 경로
input_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling"
output_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling/scaling100_noORG"

coordinates = {
    'rice': (0, 256, 320, 384),  # 하단 왼쪽
    'soup': (320, 256, 320, 384),  # 하단 오른쪽
    'side': [
        (0, 0, 160, 256),    # 상단 1구
        (160, 0, 160, 256),  # 상단 2구
        (320, 0, 160, 256),  # 상단 3구
        (480, 0, 160, 256)   # 상단 4구
    ]
}

class_folders = {
    'rice': ['01011001', '01012006', '01012002','03011011'],
    'soup': ['04011005', '04011007', '04017001', '04011011'],
    'side': ['06012004', '07014001', '11013007', '12011008', '06012008', '08011003', '10012001', '11013002', '12011003', '07013003', '11013010','08012001']
}

scale_and_save_images_with_coordinates_and_json(
    input_dir=input_directory,
    output_dir=output_directory,
    coordinates=coordinates,
    class_folders=class_folders
)


Processing folder: scaling100_noORG, Total images found: 0
Skipping unknown class folder: scaling100_noORG


### 50org+100scaled

In [ ]:
import os
import random
import cv2
import numpy as np
import json

def scale_and_save_images_with_coordinates_and_json(input_dir, output_dir, coordinates=None, class_folders=None):
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 좌표 및 클래스 정보가 없으면 예외처리
    if coordinates is None or class_folders is None:
        raise ValueError("Coordinates and class_folders must be provided.")
    
    # 캔버스 초기화
    canvas_size = (640, 640)
    # 입력 폴더 내의 각 클래스 폴더를 처리함
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)

        # 폴더가 아닌 경우 건너뜀(바로 이미지 파일이 나오면 건너뜀)
        if os.path.isdir(class_path):
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            print(f"Processing folder: {class_folder}, Total images found: {len(images)}")
            # 클래스 라벨 정하기(밥 국 반찬 중 하나)
            if class_folder in class_folders['rice']:
                class_label = 'rice'
            elif class_folder in class_folders['soup']:
                class_label = 'soup'
            elif class_folder in class_folders['side']:
                class_label = 'side'
            else:
                print(f"Skipping unknown class folder: {class_folder}")
                continue
            # 클래스별 출력 폴더 생성
            output_folder = os.path.join(output_dir, class_folder)
            os.makedirs(output_folder, exist_ok=True)
            # 모든 이미지 선택(100퍼센트 스케일링)
            selected_images = images
            print(f"Selected all {len(selected_images)} images from folder: {class_folder}")

            # 이미지를 섞고 50%를 선택하여 원본으로 저장(50org)
            random.shuffle(selected_images)
            num_original_to_save = len(selected_images) // 2
            original_to_save = selected_images[:num_original_to_save]

            for image_file in selected_images:
                image_path = os.path.join(class_path, image_file)
                json_path = os.path.join(class_path, os.path.splitext(image_file)[0] + '.json')

                # JSON 파일이 없으면 건너뜀
                if not os.path.exists(json_path):
                    print(f"No JSON file for image: {image_file}. Skipping...")
                    continue
                # 이미지 읽기
                original_image = cv2.imread(image_path)
                if original_image is None:
                    print(f"Error: Failed to read image: {image_path}. Skipping...")
                    continue

                print(f"Processing image: {image_file} with JSON: {json_path}")

                # 클래스에 따라 좌표 가져오기
                try:
                    if class_label == 'rice':
                        x, y, w, h = coordinates['rice']
                    elif class_label == 'soup':
                        x, y, w, h = coordinates['soup']
                    elif class_label == 'side':
                        x, y, w, h = random.choice(coordinates['side'])

                    # 캔버스 초기화 (흰 배경)
                    canvas = np.ones((canvas_size[1], canvas_size[0], 3), dtype=np.uint8) * 255
                    # 이미지 크기 조정 및 캔버스에 배치
                    resized_image = cv2.resize(original_image, (w, h), interpolation=cv2.INTER_AREA)
                    canvas[y:y + h, x:x + w] = resized_image

                    # 스케일링된 이미지 이름 생성 및 저장
                    new_image_name = os.path.splitext(image_file)[0] + "_scaled.jpg"
                    new_image_path = os.path.join(output_folder, new_image_name)
                    cv2.imwrite(new_image_path, canvas)
                    print(f"Saved scaled image: {new_image_path}")
                    # counter = 1
                    # while os.path.exists(new_image_path):
                    #     new_image_name = f"{os.path.splitext(image_file)[0]}_scaled_{counter}.jpg"
                    #     new_image_path = os.path.join(output_folder, new_image_name)
                    #     counter += 1


                    # JSON 파일 읽기 및 변환
                    with open(json_path, 'r') as jf:
                        json_data = json.load(jf)
                    # JSON 파일의 폴리곤 좌표 변환
                    transformed_points = []
                    if 'shapes' in json_data and len(json_data['shapes']) > 0:
                        for point in json_data['shapes'][0]['points']:
                            old_x, old_y = point
                            transformed_x = x + (old_x / original_image.shape[1]) * w
                            transformed_y = y + (old_y / original_image.shape[0]) * h
                            transformed_points.append([transformed_x, transformed_y])
                    else:
                        print(f"Skipping JSON file with missing or empty shapes: {json_path}")
                        os.remove(new_image_path)  # Delete scaled image due to JSON issue
                        print(f"Deleted scaled image due to missing JSON shapes: {new_image_path}")
                        continue

                    # 변환된 JSON 데이터 생성
                    new_json_data = {
                        "version": json_data.get("version", "0.4.15"),
                        "flags": {},
                        "shapes": [
                            {
                                "label": json_data['shapes'][0]['label'],
                                "text": "",
                                "points": transformed_points,
                                "group_id": None,
                                "shape_type": "polygon",
                                "flags": {}
                            }
                        ],
                        "imagePath": new_image_name,
                        "imageData": None,
                        "imageHeight": canvas_size[1],
                        "imageWidth": canvas_size[0]
                    }

                    # 변환된 JSON 저장
                    new_json_path = os.path.join(output_folder, os.path.splitext(new_image_name)[0] + ".json")
                    with open(new_json_path, 'w') as njf:
                        json.dump(new_json_data, njf, indent=4)
                    print(f"Saved JSON file: {new_json_path}")

                    # 원본 이미지와 JSON 저장 (선택된 경우)
                    if image_file in original_to_save:
                        original_image_path = os.path.join(output_folder, image_file)  # Keep original name
                        cv2.imwrite(original_image_path, original_image)
                        print(f"Saved original image: {original_image_path}")

                        # 원본 JSON 저장
                        original_json_name = os.path.splitext(image_file)[0] + ".json"
                        original_json_path = os.path.join(output_folder, original_json_name)
                        with open(json_path, 'r') as jf:
                            json_data = json.load(jf)
                        with open(original_json_path, 'w') as ojf:
                            json.dump(json_data, ojf, indent=4)
                        print(f"Saved original JSON file: {original_json_path}")

                except Exception as e:
                    print(f"Error during processing for {image_file}: {e}")
                    if os.path.exists(new_image_path):
                        os.remove(new_image_path)
                        print(f"Deleted scaled image due to error: {new_image_path}")
                    continue  # 다음 이미지로 이동

# 경로
input_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling"
output_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling/scaling100_org50"

# 좌표
coordinates = {
    'rice': (0, 256, 320, 384),  # Bottom left
    'soup': (320, 256, 320, 384),  # Bottom right
    'side': [
        (0, 0, 160, 256),    # Top 1
        (160, 0, 160, 256),  # Top 2
        (320, 0, 160, 256),  # Top 3
        (480, 0, 160, 256)   # Top 4
    ]
}

# 클래스폴더
class_folders = {
    'rice': ['01011001', '01012006', '01012002','03011011'],
    'soup': ['04011005', '04011007', '04017001', '04011011'],
    'side': ['06012004', '07014001', '11013007', '12011008', '06012008', '08011003', '10012001', '11013002', '12011003', '07013003', '11013010','08012001']
}

# 실행
scale_and_save_images_with_coordinates_and_json(
    input_dir=input_directory,
    output_dir=output_directory,
    coordinates=coordinates,
    class_folders=class_folders
)


### 100org + 50scaled

In [ ]:
import os
import random
import cv2
import numpy as np
import json

def scale_and_save_images_with_coordinates_and_json(input_dir, output_dir, coordinates=None, class_folders=None, seed=None):
    """
    50% 이미지를 스케일링하고, 원본 100% 이미지를 추가로 저장.
    스케일링된 이미지와 원본 이미지의 JSON 파일도 함께 저장.

    Args:
        input_dir (str): 클래스별 이미지가 저장된 폴더 경로.
        output_dir (str): 결과를 저장할 폴더 경로.
        coordinates (dict): 좌표 및 크기 {'rice': (x, y, w, h), 'soup': (x, y, w, h), 'side': [(x, y, w, h), ...]}.
        class_folders (dict): 클래스별 폴더 {'rice': [...], 'soup': [...], 'side': [...]}.
        seed (int, optional): 랜덤 시드 값. 동일한 결과를 보장하려면 지정.
    """

    if seed is not None:
        random.seed(seed)  # 랜덤 시드 설정
    
    # 출력 폴더가 없으면 생성
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    # 좌표 및 클래스 폴더가 없으면 오류 발생
    if coordinates is None or class_folders is None:
        raise ValueError("좌표나 클래스폴더가 있어야함.")

    # 캔버스 크기 (640x640)
    canvas_size = (640, 640)

    # 처리한 이미지 및 json 파일 수 카운터 초기화
    scaled_image_count = 0
    scaled_json_count = 0
    original_image_count = 0
    original_json_count = 0
    # 각 클래스 폴더 처리
    for class_folder in os.listdir(input_dir):
        class_path = os.path.join(input_dir, class_folder)
        # 폴더가 아닌 경우 건너뜀(바로 이미지파일이면 건너뛰게 됨)
        if os.path.isdir(class_path):
            # 이미지 파일 목록 필터링
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            total_images = len(images)
            print(f"Processing folder: {class_folder}, Total images found: {total_images}")

            # 클래스 라벨 확인(밥 국 반찬 중 하나여야함)
            if class_folder in class_folders['rice']:
                class_label = 'rice'
            elif class_folder in class_folders['soup']:
                class_label = 'soup'
            elif class_folder in class_folders['side']:
                class_label = 'side'
            else:
                print(f"Skipping unknown class folder: {class_folder}")
                continue
            # 클래스별 출력 폴더 생성
            output_folder = os.path.join(output_dir, class_folder)
            os.makedirs(output_folder, exist_ok=True)

            # 스케일링할 50% 이미지 선택
            scaling_images = random.sample(images, len(images) // 2)
            print(f" {len(scaling_images)}개의 이미지가 {class_folder} 폴더에서 선택됨")

            # 스케일링된 이미지 저장
            for image_file in scaling_images:
                image_path = os.path.join(class_path, image_file)
                json_path = os.path.join(class_path, os.path.splitext(image_file)[0] + '.json')
                # json 파일 없으면 건너뜀
                if not os.path.exists(json_path):
                    print(f"json파일 없음: {image_file}")
                    continue
                # 이미지 읽기
                original_image = cv2.imread(image_path)
                if original_image is None:
                    print(f"[Error!]이미지 읽어오기 실패: {image_path}")
                    continue
                # 클래스에 따른 좌표 가져오기 
                if class_label == 'rice':
                    x, y, w, h = coordinates['rice']
                elif class_label == 'soup':
                    x, y, w, h = coordinates['soup']
                elif class_label == 'side':
                    x, y, w, h = random.choice(coordinates['side'])

                # 640x640 캔버스 생성 (흰 배경)
                canvas = np.ones((canvas_size[1], canvas_size[0], 3), dtype=np.uint8) * 255

                # 이미지를 좌표에 맞게 크기 조정 및 캔버스에 배치하기
                resized_image = cv2.resize(original_image, (w, h), interpolation=cv2.INTER_AREA)
                canvas[y:y + h, x:x + w] = resized_image
                # 스케일링된 이미지 저장하기
                scaled_image_name = os.path.splitext(image_file)[0] + "_scaled.jpg"
                scaled_image_path = os.path.join(output_folder, scaled_image_name)
                cv2.imwrite(scaled_image_path, canvas)
                scaled_image_count += 1
                print(f"스케일링 이미지 저장완료 : {scaled_image_path}")

                try:
                    # json 파일 읽기 및 변환
                    with open(json_path, 'r') as jf:
                        json_data = json.load(jf)

                    # JSON 파일의 폴리곤 좌표 변환
                    transformed_points = []
                    for point in json_data['shapes'][0]['points']:
                        old_x, old_y = point
                        transformed_x = x + (old_x / original_image.shape[1]) * w
                        transformed_y = y + (old_y / original_image.shape[0]) * h
                        transformed_points.append([transformed_x, transformed_y])

                    # 새 JSON 데이터 생성
                    scaled_json_data = {
                        "version": json_data.get("version", "0.4.15"),
                        "flags": {},
                        "shapes": [
                            {
                                "label": json_data['shapes'][0]['label'],
                                "points": transformed_points,
                                "group_id": None,
                                "shape_type": "polygon",
                                "flags": {}
                            }
                        ],
                        "imagePath": scaled_image_name,
                        "imageData": None,
                        "imageHeight": canvas_size[1],
                        "imageWidth": canvas_size[0]
                    }

                    new_json_path = os.path.join(output_folder, os.path.splitext(scaled_image_name)[0] + ".json")
                    with open(new_json_path, 'w') as njf:
                        json.dump(scaled_json_data, njf, indent=4)
                    scaled_json_count += 1
                    print(f"json 파일 저장완료 : {new_json_path}")

                except Exception as e:
                    print(f"{scaled_image_name} json파일 저장 실패. 에러코드: {e}")
                    # JSON 저장 실패 시 스케일링된 이미지 삭제
                    if os.path.exists(scaled_image_path):
                        os.remove(scaled_image_path)
                        scaled_image_count -= 1
                        print(f"json파일 없으니 이미지도 삭제합니다: {scaled_image_path}")

            # 원본 100% 이미지 저장
            for image_file in images:
                image_path = os.path.join(class_path, image_file)
                json_path = os.path.join(class_path, os.path.splitext(image_file)[0] + '.json')
                # json 파일 없으면 건너뜀
                if not os.path.exists(json_path):
                    print(f"json파일 없음: {image_file}")
                    continue
                # 원본이미지 읽기
                original_image = cv2.imread(image_path)
                if original_image is None:
                    print(f"[Error!]이미지 읽기 실패: {image_path}")
                    continue

                # 원본 이미지 저장
                original_image_name = os.path.splitext(image_file)[0] + "_original.jpg"
                original_image_path = os.path.join(output_folder, original_image_name)
                cv2.imwrite(original_image_path, original_image)
                original_image_count += 1
                print(f"원본 이미지 저장: {original_image_path}")

                # 원본 JSON 저장
                with open(json_path, 'r') as jf:
                    json_data = json.load(jf)

                original_json_name = os.path.splitext(image_file)[0] + "_original.json"
                original_json_path = os.path.join(output_folder, original_json_name)
                with open(original_json_path, 'w') as ojf:
                    json.dump(json_data, ojf, indent=4)
                original_json_count += 1
                print(f"원본 json파일 저장: {original_json_path}")

    # 최종 결과 출력
    print("\n--- Summary ---")
    print(f"Total scaled images saved: {scaled_image_count}")
    print(f"Total scaled JSON files saved: {scaled_json_count}")
    print(f"Total original images saved: {original_image_count}")
    print(f"Total original JSON files saved: {original_json_count}")



# 경로
input_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling"
output_directory = "C:/Users/Sesame/food_yolo_detection/test_scaling/100org_50scaled"

# 좌표
coordinates = {
    'rice': (0, 256, 320, 384),  # 하단 왼쪽
    'soup': (320, 256, 320, 384),  # 하단 오른쪽
    'side': [
        (0, 0, 160, 256),    # 상단 1구
        (160, 0, 160, 256),  # 상단 2구
        (320, 0, 160, 256),  # 상단 3구
        (480, 0, 160, 256)   # 상단 4구
    ]
}

# 클래스폴더
class_folders = {
    'rice': ['01011001', '01012006', '01012002', '03011011'],
    'soup': ['04011005', '04011007', '04017001', '04011011'],
    'side': ['06012004', '07014001', '11013007', '12011008', '06012008', '08011003', 
             '10012001', '11013002', '12011003', '07013003', '11013010', '08012001']
}

# 실행
scale_and_save_images_with_coordinates_and_json(
    input_dir=input_directory,
    output_dir=output_directory,
    coordinates=coordinates,
    class_folders=class_folders,
    seed=42 
)